# Connecting MLOS to a C++ application

This notebook walks through connecting MLOS to a C++ application within a docker container.
We will start a docker container, and run an MLOS Agent within it. The MLOS Agent will start the actual application, and communicate with it via a shared memory channel.
In this example, the MLOS Agent controls the execution of the workloads on the application, and we will later connect to the agent to optimize the configuration of our application.

The application is a "SmartCache" similar to the one in the SmartCacheOptimization notebook, though with some more parameters to tune.
The source for this example is in the `source/Examples/SmartCache` folder.

## Building the application

To build and run the necessary components for this example you need to create and run a docker image.
To that end, open a separate terminal and go to the MLOS main folder. Within that folder, run the following commands:

1. [Build the Docker image](../../../documentation/01-Prerequisites.md#build-the-docker-image) using the [`Dockerfile`](../../../Dockerfile#mlos-github-tree-view) at the root of the repository.

    ```shell
    docker build --build-arg=UbuntuVersion=20.04 -t mlos/build:ubuntu-20.04 .
    ```

2. [Run the Docker image](../../../documentation/02-Build.md#create-a-new-container-instance) you just built.

    ```shell
    docker run -it -v $PWD:/src/MLOS --name mlos-build mlos/build:ubuntu-20.04 \
        -p 127.0.0.1:50051:50051/tcp 
    ```
    This will open a shell inside the docker container.
    We're also exposing port 50051 on the docker container to port 50051 of our host machine.
    This will allow us later to connect to the optimizer that runs inside the docker container.

3. Inside the container, [build the compiled software](../../../documentation/02-Build.md#cli-make) with `make`:

    ```sh
    make dotnet-build cmake-build
    ```

The relevant output will be at:

- Mlos.Agent.Server:

    This file corresponds to the main entry point for MLOS, written in C#. You can find the source in
    `source/Mlos.Agent.Server/MlosAgentServer.cs` and the binary at
    `out/dotnet/source/Mlos.Agent.Server/obj/AnyCPU/Mlos.Agent.Server.dll`

- SmartCache:

    This is the C++ executable that implements the SmartCache and executes some workloads.
    You can find the source in `source/Examples/SmartCache/Main.cpp` and the binary at
    `out/cmake/Release/source/Examples/SmartCache/SmartCache`

- SmartCache.SettingsRegistry:

    This is the C# code that declares the configuration options for the SmartCache component, and defines the communication
    between the the MLOS Agent and the SmartCache component. You can find the source in
    `source/Examples/SmartCache/SmartCache.SettingsRegistry/AssemblyInitializer.cs` and the binary at
    `out/dotnet/source/Examples/SmartCache/SmartCache.SettingsRegistry/obj/AnyCPU/SmartCache.SettingsRegistry.dll`
        

## Starting the MLOS Agent and executing the workloads:

Within the docker container, we can now tell the agent where the configuration options are stored, by setting the `MLOS_Settings_REGISTRY_PATH`.
Then, we can run the MLOS Agent, which will in turn run the SmartCache executable.
```sh
export MLOS_SETTINGS_REGISTRY_PATH="out/dotnet/source/Examples/SmartCache/SmartCache.SettingsRegistry/obj/AnyCPU"

tools/bin/dotnet out/dotnet/source/Mlos.Agent.Server/obj/AnyCPU/Mlos.Agent.Server.dll \
    --executable out/cmake/Release/source/Examples/SmartCache/SmartCache
```

The main loop of ``SmartCache`` contains the following:

```cpp
    for (int observations = 0; observations < 100; observations++)
    {
        // run 100 observations
        std::cout << "observations: " << observations << std::endl;

        for (int i = 0; i < 20; i++)
        {
            // run a workload 20 times
            CyclicalWorkload(2048, smartCache);
        }

        bool isConfigReady = false;
        std::mutex waitForConfigMutex;
        std::condition_variable waitForConfigCondVar;

        // Setup a callback.
        //
        // OMMITTED
        // [...]

        // Send a request to obtain a new configuration.
        SmartCache::RequestNewConfigurationMessage msg = { 0 };
        mlosContext.SendTelemetryMessage(msg);
        // wait for MLOS Agent so send a message with a new configuration
        std::unique_lock<std::mutex> lock(waitForConfigMutex);
        while (!isConfigReady)
        {
            waitForConfigCondVar.wait(lock);
        }

        config.Update();
        smartCache.Reconfigure();
    }
```

After each iteration, a TelemetryMessage is sent to the MLOS Agent, and the SmartCache blocks until it receives a new configuration to run the next workload.
By default, the agent is not connected to any optimizer, and will not change the original configuration, so the workload will just run uninterrupted.

## Starting an Optimizer
We can now also start an Optimizer service for the MLOS Agent to connect to so that we can actually optimize the parameters for this workload.
As the optimizer is running in a separate process, we need to create a new shell on the running docker container using the following command:

```shell
docker exec -it mlos-build /bin/bash
```

Within the container, we now install the Python optimizer service:
```shell
pip install -e source/Mlos.Python/
```

And run it:
```shell
start_optimizer_microservice launch --port 50051
```


## Connecting the Agent to the Optimizer
Now we can start the agent again, this time also pointing it to the optimizer:
```sh
tools/bin/dotnet out/dotnet/source/Mlos.Agent.Server/obj/AnyCPU/Mlos.Agent.Server.dll \
    --executable out/cmake/Release/source/Examples/SmartCache/SmartCache \
    --optimizer-uri http://localhost:50051
```

In [1]:
from mlos.Grpc.OptimizerMonitor import OptimizerMonitor
import grpc

In [6]:
channel = grpc.insecure_channel('172.17.0.2:50051')
optimizer_monitor = OptimizerMonitor(grpc_channel=channel)
optimizer_monitor

OptimizerMonitor(grpc_channel='172.17.0.2:50051')

In [7]:
optimizer_monitor.get_existing_optimizers()

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses"
	debug_error_string = "{"created":"@1601306607.577431000","description":"Failed to pick subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":4089,"referenced_errors":[{"created":"@1601306607.577424000","description":"failed to connect to all addresses","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":393,"grpc_status":14}]}"
>